# Scan transcript sequences for CSDE1 binding motifs

Script taken from supplemetary information of "Csde1 binds transcripts involved in protein homeostasis and controls their expression in an erythroid cell line", by Moore et al. with minimal modification.

In [1]:
import os
os.chdir("../../output/gene_modules/all_data/csde1_binding")
print("Working directory is: " + os.getcwd())
import re
from Bio import SeqIO
from Bio import SeqFeature
from Bio.SeqFeature import SeqFeature
from Bio.SeqFeature import FeatureLocation
from Bio.SeqRecord import SeqRecord

Working directory is: /home/anna/Documents/single_cell_analysis/in_vivo_SI_all/final_results_and_scripts/output/gene_modules/all_data/csde1_binding


In [21]:
#Change the next line to reflect the name of your .gb file
seq_dict = SeqIO.to_dict(SeqIO.parse("sequence.gb", "genbank"))
print("Importing " + str(len(seq_dict.keys())) + " sequences.")

Importing 4281 sequences.


In [22]:
##Create a testing dictionary for assertion statements
#Has 2 "type 1" and 2 "type 2" Csde1 binding sites
has_binding_sites = SeqRecord("TTGGGGGAAGTAATTTGGGGGGGAAACGAGTTTTTTTTTGGGGGAAGTAATTTTTGGGGGGGAAACGAGTT", id = "has_binding_sites", description = "Binding site transcript (yes), variant 5")
fakeCDS = FeatureLocation(2,10)
has_binding_sites.features.append(SeqFeature(fakeCDS, type = "CDS"))
#Has no binding sites
no_binding_sites = SeqRecord("GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG", id = "no_binding_sites", description = "No binding sites (nope), variant 2")
fakeCDS = FeatureLocation(3,9)
no_binding_sites.features.append(SeqFeature(fakeCDS, type = "CDS"))
test_list = [has_binding_sites, no_binding_sites]
testing = SeqIO.to_dict(test_list)

In [23]:
##Extract gene name from value.description
def find_gene_name(dictionary, key):
    """Extract the gene name from the description of a seqrecord object.\nUtilizes a regex which finds all
    parentheses-enclosed words followed by a comma.\nUse on a dictionary of seqrecord objects created
    with SeqIO.to_dict()."""
    value = dictionary[str(key)]
    m = re.search("\(([a-zA-Z0-9]*)\),", value.description)
    gene_name = m.group() #.group() extracts the matched sequence from match object "m"
    gene_name = gene_name.replace("(", "")
    gene_name = gene_name.replace(")", "")
    gene_name = gene_name.replace(",", "")
    return gene_name

assert find_gene_name(testing, "has_binding_sites") == "yes" #Should pass without error
assert find_gene_name(testing, "no_binding_sites") == "nope"

In [24]:
##Presence of potential binding sites
def binding_sites_present(dictionary, key, site_1 = r"[GA]{5}AAGTA[AG]", site_2 =
    r"[AG]{7}[AG]AAC[AG]{3}"):
    """Searches for two regular expressions and returns "yes" if either is found and "no" if neither is
    found.\nDefault values for "site_1" and "site_2" match the Csde1 binding sites, but other regexs may be
    provided by the user.\nUse on a dictionary of seqrecord objects created with SeqIO.to_dict()."""
    value = dictionary[str(key)]
    if re.search(site_1, str(value.seq)) or \
        re.search(site_2, str(value.seq)):
        foundsite = "yes"
    else:
        foundsite = "no"
    return foundsite

assert binding_sites_present(testing, "has_binding_sites") == "yes"
assert binding_sites_present(testing, "no_binding_sites") == "no"

In [25]:
##Store matched binding sites
def binding_site_sequence(dictionary, key, site_1 = r"[GA]{5}AAGTA[AG]", site_2 = r"[AG]{7}[AG]AAC[AG]{3}"):
    """Stores the sequence(s) of the recognized patterns as a list.\nDefault values for "site_1" and
    "site_2" match the Csde1 binding sites, but other regexs may be provided by the user.\nUse on a
    dictionary of seqrecord objects created with SeqIO.to_dict()."""
    value = dictionary[str(key)] #Extracts the corresponding value per key
    runs1 = re.findall(site_1, str(value.seq)) #Stores the sequence of the matched site
    runs2 = re.findall(site_2, str(value.seq)) #Same for site 2
    allruns = runs1 + runs2
    return allruns

assert binding_site_sequence(testing, "has_binding_sites") == ['GGGGGAAGTAA', 'GGGGGAAGTAA',
'GGGGGGGAAACGAG', 'GGGGGGGAAACGAG']
assert binding_site_sequence(testing, "no_binding_sites") == []

In [26]:
##Locations of binding sites
def binding_site_locations(dictionary, key, site_1 = r"[GA]{5}AAGTA[AG]", site_2 =
    r"[AG]{7}[AG]AAC[AG]{3}"):
    """Stores the start and stop location(s) of the recognized patterns as a list of tuples.\nDefault values
    for "site_1" and "site_2" match the Csde1 binding sites, but other regexs may be provided by the
    user.\nUse on a dictionary of seqrecord objects created with SeqIO.to_dict()."""
    run_positions=[] #Create empty list to hold location tuples
    value = dictionary[str(key)] #Extract values per key
    locations1 = re.finditer(site_1, str(value.seq)) #Finds iterable match objects
    for match in locations1:
        run_start1 = match.start() + 1 #Compensate for python's counting at 0, extract start
        run_end1 = match.end() #Extract end
        runtuple = (run_start1, run_end1)
        run_positions.append(runtuple)
        
    locations2 = re.finditer(site_2, str(value.seq))
    for match in locations2:
        run_start2 = match.start() + 1
        run_end2 = match.end()
        runtuple2 = (run_start2, run_end2)
        run_positions.append(runtuple2)
    return run_positions

assert binding_site_locations(testing, "has_binding_sites") == [(3, 13), (40, 50), (17, 30), (56, 69)]
assert binding_site_locations(testing, "no_binding_sites") == []

In [27]:
##Extracting the CDS locations
#Using biopythons lists of features within SeqRecord objects
def cds_finder(dictionary, key):
    """This function takes the location of the CDS (when defined), converts it to a string,\ngets rid of
    ambiguous characters, and returns a simple list with the start of the CDS in\nposition 0 and the end of
    the CDS in position 1."""
    value = dictionary[str(key)]
    for feature in value.features:
        if feature.type == "CDS":
            cds = str(feature.location)
            cds = cds.replace("(+)", "")
            cds = cds.replace("[", "")
            cds = cds.replace("]", "")
            cds_list = cds.split(":")
            cds_list = [int(cds_list[0]), int(cds_list[1])]
            return cds_list
            
assert cds_finder(testing,"has_binding_sites") == [2, 10]
assert cds_finder(testing, "no_binding_sites") == [3, 9]

In [28]:
##Binding site in CDS?
def utr_vs_cds(dictionary, key):
    """Determines whether the binding site falls in the 3'UTR, the CDS, the 5'UTR or spans the junction
    between them.\nReturns a list of integers corresponding to how many patterns were found in each
    region:\n5'UTR, 5'UTR-CDS-spanning, CDS, CDS-3'UTR-spanning, and 3'UTR, in that order."""
    cds_location = cds_finder(dictionary, key)
    cds_start = cds_location[0]
    cds_end = cds_location[1]
    binding_sites = binding_site_locations(dictionary, key)
    UTR_5_bound = 0
    span_5UTR_CDS = 0
    CDS_bound = 0
    span_CDS_3UTR = 0
    UTR_3_bound = 0
    for i in range(0, len(binding_sites)):
        site = binding_sites[i]
        site_start = site[0]
        site_end = site[1]
    if site_start < cds_start and site_end < cds_start:
        UTR_5_bound = UTR_5_bound + 1
    elif site_start < cds_start and site_end > cds_start:
        span_5UTR_CDS = span_5UTR_CDS + 1
    elif site_start < cds_end and site_end > cds_end:
        span_CDS_3UTR = span_CDS_3UTR + 1
    elif site_start > cds_end:
        UTR_3_bound = UTR_3_bound + 1
    else:
        CDS_bound = CDS_bound + 1
    locations_5_span5_CDS_span3_3 = [UTR_5_bound, span_5UTR_CDS, CDS_bound, span_CDS_3UTR, UTR_3_bound]
    return locations_5_span5_CDS_span3_3

In [29]:
##Fix the punctuation
def punctuation_fixer(funky):
    """Fixes the format of the output table by converting lists or tuples to a string and removing the extra
    punctuation.\nStrips away "[", "]" and single quotes."""
    funky_string = str(funky)
    funky_string = funky_string.replace("[", "")
    funky_string = funky_string.replace("]", "")
    no_more_funk = funky_string.replace("\'", "")
    return no_more_funk

assert punctuation_fixer("Free[ of] 'funk") == "Free of funk"

In [31]:
##Create dictionary with tupled values containing gene name, description, transcript length, match count, binding site sequences and positions
#Uses functions defined above, see comments there for detailed explanation
csde1 = {}
metastat_foundsite = 0
metastat_nosite = 0
metastat_5UTRbound = 0
metastat_3UTRbound = 0
metastat_CDSbound = 0
metastat_5UTR_CDS_spanner = 0
metastat_CDS_3UTR_spanner = 0
spans_5UTR_CDS = 0
spans_CDS_3UTR = 0

for key, value in seq_dict.items():
    try:
        gene_name = find_gene_name(seq_dict, key)
    except:
        print("failed ",key)
        next
    seq_length = len(value.seq)
    description = value.description.replace("Mus musculus ", "")
    description = description.replace(", mRNA", "") #Removes M. musculus and mRNA from descriptions
    description = re.sub("\(([a-zA-Z0-9]*)\).", "", description)
    cds_list = cds_finder(seq_dict, key)
    start_codon = cds_list[0]
    stop_codon = cds_list[1]
    
    if binding_sites_present(seq_dict, key) == "yes":
        allruns = binding_site_sequence(seq_dict, key)
        matchcount = len(allruns)
        runstring = punctuation_fixer(allruns)
        positions = punctuation_fixer(binding_site_locations(seq_dict, key))
        metastat_foundsite = metastat_foundsite + 1
        binding_5_CDS_3 = utr_vs_cds(seq_dict, key)
        found_in_5_UTR = binding_5_CDS_3[0]
        spans_5UTR_CDS = binding_5_CDS_3[1]
        found_in_CDS = binding_5_CDS_3[2]
        spans_CDS_3UTR = binding_5_CDS_3[3]
        found_in_3_UTR = binding_5_CDS_3[4]
        metastat_5UTRbound = metastat_5UTRbound + found_in_5_UTR
        metastat_3UTRbound = metastat_3UTRbound + found_in_3_UTR
        metastat_CDSbound = metastat_CDSbound + found_in_CDS
        metastat_5UTR_CDS_spanner = metastat_5UTR_CDS_spanner + spans_5UTR_CDS
        metastat_CDS_3UTR_spanner = metastat_CDS_3UTR_spanner + spans_CDS_3UTR

    else:
        matchcount = 0
        runstring = "None"
        positions = "NA"
        metastat_nosite = metastat_nosite + 1
        found_in_5_UTR = 0
        found_in_CDS = 0
        found_in_3_UTR = 0

    #The .title() method capitalizes the first letter of each word, for aesthetic reasons
    entry = (gene_name, description.title(), matchcount, runstring, positions, found_in_5_UTR,
    found_in_CDS, found_in_3_UTR, spans_5UTR_CDS, spans_CDS_3UTR, start_codon, stop_codon,
    seq_length)
    
    print(key + " " + str(entry))
    csde1[key] = entry
    
    
total_transcripts = metastat_foundsite + metastat_nosite
print("Transcripts scanned: %d" % total_transcripts)
print("Found site in %d transcripts." % metastat_foundsite)
print("No site in %d transcripts." % metastat_nosite)
print("Found %d sites in 5'UTR." % metastat_5UTRbound)
print("Found %d sites in 3'UTR." % metastat_3UTRbound)
print("Found %d sites in CDS." % metastat_CDSbound)
print("Found %d sites spanning 5'UTR and CDS." % metastat_5UTR_CDS_spanner)
print("Found %d sites spanning CDS and 3'UTR." % metastat_CDS_3UTR_spanner)

##Write output
#Change the name of the outfile to whatever you want
outfile = open("CSDE1_binding_sites.csv", "w")
header = "Refseq ID" + "\t" + "Gene Name" + "\t" + "Full Name" + "\t" + "Number of Sites" + "\t" + "Site Sequence" + "\t" + "Site Position" + "\t" + "5' UTR Matches" + "\t"\
+ "CDS Matches" + "\t" + "3' UTR Matches" + "\t" "Match Spans 5'UTR-CDS" + "\t" + "Match Spans CDS-3'UTR" + "\t" + "Start Codon" +\
"\t" + "Stop Codon" + "\t" + "mRNA Length" + "\n"
outfile.write("Transcripts scanned: %d \n" % total_transcripts)
outfile.write("Found site in %d transcripts. \n" % metastat_foundsite)
outfile.write("No site in %d transcripts. \n" % metastat_nosite)
outfile.write("Found %d sites in 5'UTR. \n" % metastat_5UTRbound)
outfile.write("Found %d sites in 3'UTR. \n" % metastat_3UTRbound)
outfile.write("Found %d sites in CDS. \n" % metastat_CDSbound)
outfile.write("Found %d sites spanning 5'UTR and CDS. \n" % metastat_5UTR_CDS_spanner)
outfile.write("Found %d sites spanning CDS and 3'UTR.\n \n" % metastat_CDS_3UTR_spanner)
outfile.write(header)

for key, value in csde1.items():
    out_row = key
    for i in range(0, (len(value))):
        out_row = out_row + "\t" + str(value[i])
    print("Writing line: " + "\n" + out_row + "\n")
    outfile.write(out_row + "\n")
outfile.close()


NM_009215.1 ('Sst', 'Somatostatin (Sst)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 100, 451, 599)
NM_008655.1 ('Gadd45b', 'Growth Arrest And Dna-Damage-Inducible 45 Beta (Gadd45B)', 1, 'AAAAAAAGTAA', '(754, 764)', 0, 0, 1, 0, 0, 225, 708, 1305)
NM_008227.1 ('Hcn3', 'Hyperpolarization-Activated, Cyclic Nucleotide-Gated K+ 3 (Hcn3)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 98, 2438, 3268)
NM_007836.1 ('Gadd45a', 'Growth Arrest And Dna-Damage-Inducible 45 Alpha (Gadd45A)', 1, 'AAGGAAAGTAA', '(1044, 1054)', 0, 0, 1, 0, 0, 163, 661, 1224)
NM_009785.1 ('Cacna2d3', 'Calcium Channel, Voltage-Dependent, Alpha2/Delta Subunit 3 (Cacna2D3)', 1, 'AGGAAAAAAACGGA', '(1732, 1745)', 0, 1, 0, 0, 0, 97, 3373, 3710)
NM_010260.1 ('Gbp2', 'Guanylate Binding Protein 2 (Gbp2)', 1, 'AGAAGAAGTAA', '(1421, 1431)', 0, 1, 0, 0, 0, 170, 1940, 2471)
NM_011825.1 ('Grem2', 'Gremlin 2, Dan Family Bmp Antagonist (Grem2)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 248, 755, 3745)
NM_011776.1 ('Zp3', 'Zona Pellucida Glycoprotein 3 (Zp3)', 0, 'None'

NM_013657.5 ('Ig', 'Sema Domain, Immunoglobulin Domain  Short Basic Domain, Secreted, 3C (Sema3C)', 1, 'GGAAAAAGAACAGA', '(4109, 4122)', 0, 0, 1, 0, 0, 188, 2444, 4956)
NM_133888.2 ('Smpdl3b', 'Sphingomyelin Phosphodiesterase, Acid-Like 3B (Smpdl3B)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 126, 1497, 1925)
NM_013743.2 ('Pdk4', 'Pyruvate Dehydrogenase Kinase, Isoenzyme 4 (Pdk4)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 116, 1355, 3453)
NM_013874.2 ('Dpf1', 'D4, Zinc And Double Phd Fingers Family 1  Transcript Variant 1', 2, 'GAGGAAGAAACGGA, GGAGGAAGAACAGG', '(313, 326), (570, 583)', 0, 1, 0, 0, 0, 76, 1243, 2278)
NM_133726.2 ('St13', 'Suppression Of Tumorigenicity 13 (St13)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 100, 1216, 1622)
NM_008182.3 ('Gsta2', 'Glutathione S-Transferase, Alpha 2 (Gsta2)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 79, 748, 855)
NM_013900.2 ('Meltf', 'Melanotransferrin (Meltf)', 2, 'AAAAAAAGTAA, GGAGAAGAAACAGG', '(3589, 3599), (3610, 3623)', 0, 0, 1, 0, 0, 117, 2334, 4133)
NM_015731.3 ('Atp9a',

NM_053080.3 ('Aldh1a3', 'Aldehyde Dehydrogenase Family 1, Subfamily A3 (Aldh1A3)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 65, 1604, 3423)
NM_133221.2 ('Slc8b1', 'Solute Carrier Family 8 (Sodium/Lithium/Calcium Exchanger), Member B1  Transcript Variant 1', 0, 'None', 'NA', 0, 0, 0, 0, 0, 149, 1907, 2893)
NM_016671.3 ('Il27ra', 'Interleukin 27 Receptor, Alpha (Il27Ra)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 41, 1913, 2671)
NM_028039.2 ('Esco2', 'Establishment Of Sister Chromatid Cohesion N-Acetyltransferase 2 (Esco2)', 1, 'GAAAAGAGAACGGG', '(1441, 1454)', 0, 1, 0, 0, 0, 181, 1960, 2899)
NM_133710.3 ('Ctdspl', 'Ctd (Carboxy-Terminal Domain, Rna Polymerase Ii, Polypeptide A) Small Phosphatase-Like (Ctdspl)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 326, 1157, 4627)
NM_010336.2 ('Lpar1', 'Lysophosphatidic Acid Receptor 1  Transcript Variant 1', 0, 'None', 'NA', 0, 0, 0, 0, 0, 186, 1281, 3362)
NM_172989.1 ('Lpar1', 'Lysophosphatidic Acid Receptor 1  Transcript Variant 2', 0, 'None', 'NA', 0, 0, 0, 0, 0, 275, 1370

NM_001159567.1 ('Meis2', 'Meis Homeobox 2  Transcript Variant 3', 0, 'None', 'NA', 0, 0, 0, 0, 0, 245, 1451, 4737)
NM_001159568.1 ('Meis2', 'Meis Homeobox 2  Transcript Variant 4', 0, 'None', 'NA', 0, 0, 0, 0, 0, 245, 1679, 4642)
NM_018761.3 ('Ctnnal1', 'Catenin (Cadherin Associated Protein), Alpha-Like 1 (Ctnnal1)', 1, 'AGAGAAAGTAA', '(637, 647)', 0, 1, 0, 0, 0, 97, 2293, 3712)
NM_019436.3 ('Sit1', 'Suppression Inducing Transmembrane Adaptor 1 (Sit1)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 126, 669, 1201)
NM_008792.4 ('Pcsk2', 'Proprotein Convertase Subtilisin/Kexin Type 2 (Pcsk2)', 1, 'AAAAGAAAAACAGA', '(243, 256)', 1, 0, 0, 0, 0, 358, 2272, 4766)
NM_028132.3 ('Pgm1', 'Phosphoglucomutase 1 (Pgm1)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 115, 1804, 2334)
NM_015747.2 ('Slc20a1', 'Solute Carrier Family 20, Member 1  Transcript Variant 1', 0, 'None', 'NA', 0, 0, 0, 0, 0, 490, 2536, 3316)
NM_001159593.1 ('Slc20a1', 'Solute Carrier Family 20, Member 1  Transcript Variant 2', 0, 'None', 'NA', 0, 0, 0, 0

NM_001033247.1 ('Cfap44', 'Cilia And Flagella Associated Protein 44 (Cfap44)', 1, 'GAAGAAAGTAG', '(2827, 2837)', 0, 1, 0, 0, 0, 55, 5587, 6583)
NM_030709.2 ('Tmprss5', 'Transmembrane Protease, Serine 5  Transcript Variant 2', 0, 'None', 'NA', 0, 0, 0, 0, 0, 115, 1453, 2070)
NM_024245.4 ('Kif23', 'Kinesin Family Member 23 (Kif23)', 2, 'AGAAGAAAAACAAA, AAAAAAAAAACAAA', '(1865, 1878), (3213, 3226)', 0, 0, 1, 0, 0, 177, 3039, 3439)
NM_023729.3 ('Asz1', 'Ankyrin Repeat, Sam And Basic Leucine Zipper Domain Containing 1 (Asz1)', 1, 'AAGAGAAGAACGAG', '(156, 169)', 0, 1, 0, 0, 0, 25, 1453, 1740)
NM_026018.3 ('Pdzk1ip1', 'Pdzk1 Interacting Protein 1  Transcript Variant 2', 0, 'None', 'NA', 0, 0, 0, 0, 0, 206, 551, 897)
NM_001164557.1 ('Pdzk1ip1', 'Pdzk1 Interacting Protein 1  Transcript Variant 1', 0, 'None', 'NA', 0, 0, 0, 0, 0, 237, 783, 1129)
NM_001164558.1 ('Pdzk1ip1', 'Pdzk1 Interacting Protein 1  Transcript Variant 3', 0, 'None', 'NA', 0, 0, 0, 0, 0, 135, 480, 826)
NM_026507.4 ('Zwilch', '

...

NM_199239.3 ('TM', 'Sema Domain, Transmembrane Domain  And Cytoplasmic Domain, 6D  Transcript Variant 5', 0, 'None', 'NA', 0, 0, 0, 0, 0, 54, 3147, 5627)
NM_001290982.1 ('Myo1b', 'Myosin Ib  Transcript Variant 3', 0, 'None', 'NA', 0, 0, 0, 0, 0, 227, 3464, 4879)
NM_001291000.1 ('TM', 'Sema Domain, Transmembrane Domain  And Cytoplasmic Domain, 6D  Transcript Variant 7', 0, 'None', 'NA', 0, 0, 0, 0, 0, 819, 4029, 6509)
NM_010863.4 ('Myo1b', 'Myosin Ib  Transcript Variant 2', 0, 'None', 'NA', 0, 0, 0, 0, 0, 227, 3551, 4966)
NM_172537.4 ('TM', 'Sema Domain, Transmembrane Domain  And Cytoplasmic Domain, 6D  Transcript Variant 1', 0, 'None', 'NA', 0, 0, 0, 0, 0, 819, 3855, 6335)
NM_001161817.2 ('Myo1b', 'Myosin Ib  Transcript Variant 1', 0, 'None', 'NA', 0, 0, 0, 0, 0, 227, 3638, 5053)
NM_199241.3 ('TM', 'Sema Domain, Transmembrane Domain  And Cytoplasmic Domain, 6D  Transcript Variant 4', 0, 'None', 'NA', 0, 0, 0, 0, 0, 54, 3276, 5756)
NM_199240.3 ('TM', 'Sema Domain, Transmembrane Domain  

NM_007886.3 ('Dtnb', 'Dystrobrevin, Beta  Transcript Variant 2', 1, 'GAAGAGGAAACGAA', '(4732, 4745)', 0, 0, 1, 0, 0, 250, 2059, 5055)
NM_016696.5 ('Gpc1', 'Glypican 1 (Gpc1)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 287, 1961, 3608)
NM_001162465.2 ('Dtnb', 'Dystrobrevin, Beta  Transcript Variant 1', 1, 'GAAGAGGAAACGAA', '(4704, 4717)', 0, 0, 1, 0, 0, 258, 2238, 5027)
NM_001289493.2 ('Gbp3', 'Guanylate Binding Protein 3  Transcript Variant 3', 1, 'AAAGAAAGTAA', '(2334, 2344)', 0, 0, 1, 0, 0, 359, 2222, 2743)
NM_009828.3 ('Ccna2', 'Cyclin A2 (Ccna2)', 0, 'None', 'NA', 0, 0, 0, 0, 0, 738, 2007, 3376)
NM_028829.4 ('Paqr8', 'Progestin And Adipoq Receptor Family Member Viii  Transcript Variant 1', 0, 'None', 'NA', 0, 0, 0, 0, 0, 361, 1426, 5404)
NM_001355254.1 ('Snap25', 'Synaptosomal-Associated Protein 25  Transcript Variant 3', 0, 'None', 'NA', 0, 0, 0, 0, 0, 160, 781, 2066)
NM_001355053.1 ('Bcl2l1', 'Bcl2-Like 1  Transcript Variant 5', 0, 'None', 'NA', 0, 0, 0, 0, 0, 462, 1164, 2645)
NM_001355170

NM_009947.4 ('Cpne6', 'Copine Vi  Transcript Variant 2', 0, 'None', 'NA', 0, 0, 0, 0, 0, 92, 1766, 2008)
NM_001136057.3 ('Cpne6', 'Copine Vi  Transcript Variant 1', 0, 'None', 'NA', 0, 0, 0, 0, 0, 292, 1966, 2208)
NM_001146183.2 ('Cpne6', 'Copine Vi  Transcript Variant 3', 0, 'None', 'NA', 0, 0, 0, 0, 0, 269, 1940, 2182)
NM_001360193.1 ('Cpne6', 'Copine Vi  Transcript Variant 4', 0, 'None', 'NA', 0, 0, 0, 0, 0, 269, 1943, 2185)
NM_001360269.1 ('Ndrg2', 'N-Myc Downstream Regulated Gene 2  Transcript Variant 8', 0, 'None', 'NA', 0, 0, 0, 0, 0, 41, 1157, 2026)
NM_001360276.1 ('Ndrg2', 'N-Myc Downstream Regulated Gene 2  Transcript Variant 13', 0, 'None', 'NA', 0, 0, 0, 0, 0, 87, 1161, 2030)
NM_001360273.1 ('Ndrg2', 'N-Myc Downstream Regulated Gene 2  Transcript Variant 12', 0, 'None', 'NA', 0, 0, 0, 0, 0, 171, 1245, 2114)
NM_001360318.1 ('Htr7', '5-Hydroxytryptamine Receptor 7  Transcript Variant 3', 1, 'AGAAGAAGTAA', '(1478, 1488)', 0, 0, 1, 0, 0, 107, 1415, 3398)
NM_001360267.1 ('Ndrg2'

Writing line: 
NM_013874.2	Dpf1	D4, Zinc And Double Phd Fingers Family 1  Transcript Variant 1	2	GAGGAAGAAACGGA, GGAGGAAGAACAGG	(313, 326), (570, 583)	0	1	0	0	0	76	1243	2278

Writing line: 
NM_133726.2	St13	Suppression Of Tumorigenicity 13 (St13)	0	None	NA	0	0	0	0	0	100	1216	1622

Writing line: 
NM_008182.3	Gsta2	Glutathione S-Transferase, Alpha 2 (Gsta2)	0	None	NA	0	0	0	0	0	79	748	855

Writing line: 
NM_013900.2	Meltf	Melanotransferrin (Meltf)	2	AAAAAAAGTAA, GGAGAAGAAACAGG	(3589, 3599), (3610, 3623)	0	0	1	0	0	117	2334	4133

Writing line: 
NM_015731.3	Atp9a	Atpase, Class Ii, Type 9A  Transcript Variant 1	0	None	NA	0	0	0	0	0	51	3195	3529

Writing line: 
NM_133662.2	Ier3	Immediate Early Response 3 (Ier3)	0	None	NA	0	0	0	0	0	6	489	1090

Writing line: 
NM_008113.3	Arhgdig	Rho Gdp Dissociation Inhibitor Gamma (Arhgdig)	0	None	NA	0	0	0	0	0	207	885	1032

Writing line: 
NM_001037906.2	Nell1	Nel-Like 1 (Nell1)	0	None	NA	0	0	0	0	0	39	2472	2812

Writing line: 
NM_008329.2	Ifi204	Interferon Activa

Writing line: 
NM_001293561.1	Unc5c	Unc-5 Netrin Receptor C  Transcript Variant 1	1	AGAAAAAGTAA	(7029, 7039)	0	0	1	0	0	150	3003	9355

Writing line: 
NM_009472.4	Unc5c	Unc-5 Netrin Receptor C  Transcript Variant 2	1	AGAAAAAGTAA	(6972, 6982)	0	0	1	0	0	150	2946	9298

Writing line: 
NM_001293644.1	Mcub	Mitochondrial Calcium Uniporter Dominant Negative Beta Subunit  Transcript Variant 2	0	None	NA	0	0	0	0	0	131	890	1171

Writing line: 
NM_001293683.1	Zwint	Zw10 Interactor  Transcript Variant 2	0	None	NA	0	0	0	0	0	73	832	1851

Writing line: 
NM_025635.4	Zwint	Zw10 Interactor  Transcript Variant 1	0	None	NA	0	0	0	0	0	73	832	2729

Writing line: 
NM_001293684.1	Zwint	Zw10 Interactor  Transcript Variant 3	0	None	NA	0	0	0	0	0	73	832	1816

Writing line: 
NM_011281.3	Rorc	Rar-Related Orphan Receptor Gamma  Transcript Variant 1	0	None	NA	0	0	0	0	0	98	1649	2572

Writing line: 
NM_001293734.1	Rorc	Rar-Related Orphan Receptor Gamma  Transcript Variant 2	0	None	NA	0	0	0	0	0	165	1653	2576

Writing line: 
